In [3]:
import sys 

sys.path.append(r'H:\TUM-PC\Dokumente\Projects\AMfe')
import amfe
import matplotlib.pyplot as plt
import numpy as np
import scipy

msh_dict = {}
msh_dict[0] = amfe.amfe_dir('meshes/test_meshes/Geom3.msh')
msh_dict[1] = amfe.amfe_dir('meshes/test_meshes/simple_2.msh')
msh_dict[2] = mshfile = amfe.amfe_dir('meshes/test_meshes/3_partition_pressure_corner.msh')
msh_dict[3] = mshfile = amfe.amfe_dir('meshes/test_meshes/geo_hole_quad_part_4.msh')
msh_dict[4] = mshfile = amfe.amfe_dir('meshes/test_meshes/2_partitions_quad_mesh.msh')
msh_dict[5] = mshfile = amfe.amfe_dir('meshes/test_meshes/2_partitions_2quad_mesh.msh')
msh_dict[6] = mshfile = amfe.amfe_dir('meshes/test_meshes/4_partitions_quad_mesh.msh')
msh_dict[7] = mshfile = amfe.amfe_dir('meshes/test_meshes/3_partition_2d_blade_quad_mesh.msh')


domain_id = {}
domain_id[0] = 3
domain_id[1] = 3
domain_id[2] = 11
domain_id[3] = 8
domain_id[4] = 3
domain_id[5] = 3
domain_id[6] = 3
domain_id[7] = 3

# select mesh to be plotted
mesh_id = 0
mshfile = msh_dict[mesh_id]

m = amfe.Mesh()
m.import_msh(mshfile)


# creating material
my_material = amfe.KirchhoffMaterial(E=210E9, nu=0.3, rho=7.86E3, plane_stress=True, thickness=0.1)


# splitting physical grops

m.split_in_groups()

# plotting boundary elements
amfe.plot_boundary_1d(m)



# plotting mesh
amfe.plot_submesh(m.groups[domain_id[mesh_id]])


# setting boundary condition
# selecting subdomain for boundary condition
sub_dir = m.get_submesh('physical',1)
sub_neu = m.get_submesh('physical',2)

value = 5.0E9
neu = amfe.boundary.Boundary(sub_neu,value,'normal')
diri = amfe.boundary.Boundary(sub_dir,0,'xy','dirichlet')


# setting main domain for FE calculation
domain = m.set_domain('phys_group', domain_id[mesh_id])
domain.set_material(my_material)
domain.append_bondary_condition(neu)
domain.append_bondary_condition(diri)
domain.split_in_partitions()

amfe.plot_domain(domain)

plt.show()

ModuleNotFoundError: No module named 'h5py'

Solving the Standard FE problem
---------------------------

$$
Ku = f
$$


In [2]:
scale = 1
my_system = amfe.MechanicalSystem()
my_system.set_mesh_obj(m)
my_system.set_domain(3,my_material)
my_system.apply_dirichlet_boundaries(1, 'xy')
my_system.apply_neumann_boundaries(2, value, 'normal')

s = amfe.LinearStaticsSolver(my_system)
s.solve()

u_fea = my_system.u_output[1]
connectivity = my_system.mesh_class.connectivity
nodes = my_system.mesh_class.nodes


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_static(scale):
    amfe.plotDeformQuadMesh(connectivity,nodes,my_system.u_output[1],scale) 
    plt.show()
    
interact(plot_static,mode=(0,10,1),scale=(0,10,1))  



 phys_group 3 with 1102 elements successfully added.
Total number of elements in mesh: 1102
*************************************************************
Preallocating the stiffness matrix
Done preallocating stiffness matrix with 1102 elements and 2360 dofs.
Time taken for preallocation: 0.02 seconds.

 phys_group 1 with 20 nodes successfully added to Dirichlet Boundaries.
Total number of nodes with Dirichlet BCs: 20
Total number of constrained dofs: 40
*************************************************************

 phys_group 2 with 19 elements successfully added to Neumann Boundary.
Total number of neumann elements in mesh: 19
Total number of elements in mesh: 1102
*************************************************************
Attention: No linear solver was given, setting linear_solver = PardisoSolver.
Attention: No pseudo time evaluation was given, setting t = 1.0.
Assembling external force and stiffness...


C:\Users\ge72tih\AppData\Local\Continuum\anaconda3\envs\py36\Lib\site-packages\scipy\sparse\sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


Start solving linear static problem...
Static problem solved.


A Jupyter Widget

<function __main__.plot_static>

Solving with Dual Assembly
---------------------------

$$
\begin{bmatrix} K & B^{T} \\
                 B & 0  
\end{bmatrix}
\begin{bmatrix} u \\ 
\lambda \end{bmatrix}
=
\begin{bmatrix} f \\ 
0 \end{bmatrix}
$$



In [3]:
# creating FETI dict subdomain
sub1 = amfe.FETIsubdomain(domain.groups[1])
sub2 = amfe.FETIsubdomain(domain.groups[2])

feti_subdomain = {}
feti_subdomain[1] = sub1
feti_subdomain[2] = sub2

K1, f = sub1.assemble_k_and_f()
K1 = K1.todense()
K, fext1 = sub1.assemble_k_and_f_neumann()
K1, f1 = sub1.insert_dirichlet_boundary_cond(K1,fext1)
B1_dict = sub1.assemble_interface_boolean_matrix()



K2, f = sub2.assemble_k_and_f()
K2 = K2.todense()
K, fext2 = sub2.assemble_k_and_f_neumann()
B2_dict = sub2.assemble_interface_boolean_matrix()


B1 = B1_dict[1,2].todense()
B2 = B2_dict[2,1].todense()

# Assemble the block subdomain matrix
n1 = K1.shape[0]
n2 = K2.shape[0]
nlambda = B1.shape[0]
u_dof = n1 + n2
total_dof = u_dof + nlambda

# assemble boolean matrix
B = np.hstack((B1,B2))

# assemble K matrix
Kd = scipy.linalg.block_diag(K1,K2)
Zero_block = np.zeros([nlambda,nlambda])

D1 = np.hstack((Kd,B.T))
D2 = np.hstack((B,Zero_block))
D = np.vstack((D1,D2))


fd = np.hstack((fext1,fext2))
zero_vector = np.zeros(nlambda)
b = np.hstack((fd,zero_vector))


ax1 = plt.subplot(231)
ax1.imshow(Kd, interpolation='nearest')
plt.axis('off')

ax2 = plt.subplot(232)
ax2.imshow(B.T, interpolation='nearest')
plt.axis('off')

ax3 = plt.subplot(234)
ax3.imshow(B, interpolation='nearest')
plt.axis('off')


ax4 = plt.subplot(235)
ax4.imshow(Zero_block, interpolation='nearest')
plt.axis('off')

ax5 = plt.subplot(233)
ax5.imshow(np.matrix(fd).T, interpolation='nearest')
plt.axis('off')

ax6 = plt.subplot(236)
ax6.imshow(np.matrix(zero_vector).T, interpolation='nearest')
plt.axis('off')

plt.show()

Preallocating the stiffness matrix
Done preallocating stiffness matrix with 367 elements and 814 dofs.
Time taken for preallocation: 0.01 seconds.
Preallocating the stiffness matrix
Done preallocating stiffness matrix with 367 elements and 816 dofs.
Time taken for preallocation: 0.01 seconds.


IndexError: index 1 is out of bounds for axis 1 with size 1

Generally the block matrix $K$ is singular due to local rigid body modes, then the inner problem is regularized by adding a subset of the inter-subdomain compatibility requeriments:


$$
\begin{bmatrix} K & G^{T} & B^{T} \\
                G & 0 & 0   \\
                B & 0 & 0   \\
\end{bmatrix}
\begin{bmatrix} u \\ 
\alpha \\
\lambda \end{bmatrix}
=
\begin{bmatrix} f \\ 
0 \\
0 \end{bmatrix}
$$

Where $G$ is defined as $-R^TB^T$.

In [ ]:
# eingen spectron of the operators
w, V = np.linalg.eigh(Kd)

w_id = np.argsort(-w)
w= w[w_id]
V = V[:,w_id]


cond_num_Kd = w[0]/w[-1]

plt.ylim([0.0,np.max(w)])
plt.bar(np.arange(len(w)),w)
plt.title('Eigenvalues of block matrix K, $\kappa = $ ' + '{0:.2e}'.format(cond_num_Kd))

plt.show()

Solving Equilibrium with Dual Interface Assembly Problem
----------------------------------------------

The Dual Assembly system of equation discribe above can be broken in two equations.

\begin{equation}
Ku + B^{T}\lambda  = f \\
Bu = 0 
\end{equation}

Then, the solution u can be calculate by:

\begin{equation}
u =  K^*(B^{T}\lambda  - f) +  R\alpha \\
\end{equation}

Where $K^*$ is the generelize pseudo inverse and $R$ is $Null(K) = \{r \in R: Kr=0\}$, named the kernel of the K matrix.
In order to the solve $u$ the summation of all forces in the subdomain, interface, internal and extenal forces must be in the image of K. This implies the $(B^{T}\lambda  + f)$ must be orthonal to the null space of K.

\begin{equation}
R^T(B^{T}\lambda  - f) = 0 \\
\end{equation}

Phisically, the equation above eforces the self-equilibrim for each subdomain. Using the compatibility equation and the self-equilibrium equation, we can write the dual interface equilibrium equation as:


$$
\begin{bmatrix} F & G^{T} \\
                 G & 0  
\end{bmatrix}
\begin{bmatrix} \lambda  \\ 
\alpha
\end{bmatrix}
=
\begin{bmatrix} d \\ 
e \end{bmatrix}
$$

Where $F = BK^*B^T$, $G = -R^TB^T$, $d = BK^*f$ and $e =- R^Tf $.


In [ ]:
# assemple dual interface operator F and G
feti_sub = {}
feti_sub[1] = sub1
feti_sub[2] = sub2

lambda_dict = {}
alpha_dict = {}
F_dict = {}
G_dict = {}
e_dict = {}
d_dict = {}
init_dof = 0
init_alpha = 0
interface_size = 0
null_space_size = 0
for sub_key in feti_sub:
    sub = feti_sub[sub_key]
    sub.create_interface_and_interior_dof_dicts()
    
    B_dict = sub.assemble_interface_boolean_matrix()
    
    for nei_key in sub.local_interface_dofs_dict:
        
        if (nei_key,sub_key) not in lambda_dict:
            last_dof = len(sub.local_interface_dofs_dict[nei_key])
            interface_size +=last_dof
            lambda_dict[(sub_key,nei_key)] = np.arange(init_dof,init_dof+last_dof)
            init_dof = last_dof + 1
            sub_nei = feti_sub[nei_key]
            Bnei_dict = sub_nei.assemble_interface_boolean_matrix()
            Bnei = Bnei_dict[nei_key,sub_key].todense()
            
        else:
            lambda_dict[(sub_key,nei_key)] =  lambda_dict[(nei_key,sub_key)]

        Kinv, R = sub.calc_pinv_and_null_space()
        Bsub = B_dict[sub_key,nei_key].todense()
        Bij = Bsub
        Fij = Bij.dot(Kinv).dot(Bij.T)
        F_dict[sub_key,nei_key] = Fij
        
        f = sub.force
        d = Bij.dot(Kinv).dot(f)
        
        d_dict[(sub_key,nei_key)] =  d
        
        last_alpha = R.shape[1]
        null_space_size += last_alpha
        
        if last_alpha>0:
            alpha_dict[(sub_key)] = np.arange(init_alpha,init_alpha+last_alpha)
            init_alpha = last_alpha
            Gij = -Bij.dot(R)
            G_dict[sub_key,nei_key] = Gij
            e = -R.T.dot(f)
            e_dict[(sub_key,nei_key)] = e
            init_alpha = last_alpha
            
            
        
F = np.zeros([interface_size,interface_size])
G = np.zeros([interface_size,null_space_size])
Zeros = np.zeros([null_space_size,null_space_size])
d = np.zeros([interface_size])
e = np.zeros([null_space_size])
for sub_key,nei_key in F_dict:
    if sub_key<nei_key:
        for loc_i,i in enumerate(lambda_dict[(sub_key,nei_key)]):
            d[i] = d_dict[(sub_key,nei_key)][loc_i] + d_dict[(nei_key,sub_key)][loc_i]
            for loc_j,j in enumerate(lambda_dict[(nei_key,sub_key)]):
                F[i,j] =  F_dict[(sub_key,nei_key)][loc_i,loc_j] + F_dict[(nei_key,sub_key)][loc_i,loc_j]

                               
for sub_key,nei_key in G_dict:
        for loc_i,i in enumerate(alpha_dict[(sub_key)]):
            e[i] = float(e_dict[(sub_key,nei_key)][loc_i])
            for loc_j,j in enumerate(lambda_dict[(sub_key,nei_key)]):
                G[j,i] =  G_dict[(sub_key,nei_key)][loc_j,loc_i] 
                
G = G.T 
                                                
A1 = np.hstack((F,G.T))
A2 = np.hstack((G,Zeros))

A = np.vstack((A1,A2))
b = np.concatenate((d,e))

ax1 = plt.subplot(231)
ax1.imshow(F, interpolation='nearest')
plt.axis('off')

ax2 = plt.subplot(232)
ax2.imshow(G, interpolation='nearest')
plt.axis('off')

ax3 = plt.subplot(234)
ax3.imshow(G.T, interpolation='nearest')
plt.axis('off')


ax4 = plt.subplot(235)
ax4.imshow(Zeros, interpolation='nearest')
plt.axis('off')

ax5 = plt.subplot(233)
ax5.imshow(np.matrix(d).T, interpolation='nearest')
plt.axis('off')

ax6 = plt.subplot(236)
ax6.imshow(np.matrix(e).T, interpolation='nearest')
plt.axis('off')

plt.show()

# Dual interface problem
x = np.linalg.solve(A,b)
global_lambda_int = x[:interface_size]
alpha = x[interface_size:]





In [ ]:
global_lambda_int

In [ ]:
global_lambda = np.matrix(global_lambda_int).T
global_alpha = np.matrix(alpha).T
u1 = sub1.solve_local_displacement(1.0*global_lambda,lambda_dict,'svd')
u1 = sub1.apply_rigid_body_correction(global_alpha,alpha_dict)
u1 = np.array(u1).flatten()


u2 = sub2.solve_local_displacement(1.0*global_lambda,lambda_dict,'svd')
u2 = sub2.apply_rigid_body_correction(global_alpha,alpha_dict)
u2 = np.array(u2).flatten()

connectivity = sub1.mesh.connectivity
nodes = sub1.mesh.nodes

connectivity2 = sub2.mesh.connectivity
nodes2 = sub2.mesh.nodes


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_static(scale):
    tri, ax = amfe.plotDeformQuadMesh(connectivity,nodes,u1,scale) 
    tri, ax = amfe.plotDeformQuadMesh(connectivity2,nodes2,u2,scale,ax) 
    plt.show()
    
interact(plot_static,mode=(0,10,1),scale=(0,10,1))  



In [ ]:
global_lambda

In [ ]:
u_dual_int = amfe.FetiSolver.average_displacement_calc(my_system,feti_subdomain)

residual = abs(u_fea - u_dual_int)
p , residuals, rank, singular_values, rcond = np.polyfit(u_fea, u_dual_int, 1, full=True)
m = p[0]
b = p[1]


fig, axs = plt.subplots(1, 2, figsize=(10, 4))
plt.tight_layout(pad=4.0, w_pad=4.5, h_pad=4.0)

axs[0].bar(np.arange(len(residual)),residual)
axs[0].set_title('Displacement error per DoFs')
axs[0].set_xlabel('DoF number')
axs[0].set_ylabel('Error')

axs[1].scatter(u_fea, u_dual_int)
X_plot = np.linspace(axs[1].get_xlim()[0],axs[1].get_xlim()[1],100)
axs[1].plot(X_plot, m*X_plot + b, '-')
correlation = np.corrcoef(u_fea, u_dual_int)[0,1]
axs[1].set_title('Correlation between $u$ calc by Primal and Dual Interface')
axs[1].text(0.01, 0.3, r"$r^2 = $" + '{0:.2f}'.format(correlation**2) , fontsize=20)
axs[1].set_xlabel('Primal Assembly $u$ ')
axs[1].set_ylabel('Dual interface $u$')

fig.suptitle('Error between Primal and Dual Assembly')
plt.show()



In [ ]:
connectivity = my_system.mesh_class.connectivity
nodes = my_system.mesh_class.nodes


def plot_static(scale):
    amfe.plotDeformQuadMesh(connectivity,nodes,u_dual_int,scale) 
    plt.show()
    
interact(plot_static,mode=(0,10,1),scale=(0,10,1))  

Solving Equilibrium with The Projected Dual Interface 
---------------------------------------------------------

The two equation that must be solve in Dual interface problem are:

\begin{equation}
F\lambda + G^{T}\alpha  = d \\
G\lambda = e 
\end{equation}


Let's define now the Projector operator $P_G$ in to the $Ker(G)$ as:

\begin{equation}
P_G = I - Q_GG = I - G^T(GG^T)^{-1}G
\end{equation}

Where $ P_G(\lambda) = \{ \lambda_{kernel} = P_G\lambda : \lambda_{kernel} \in Ker(G) \}$.
We also define $ Q_G(\lambda) = \{ \lambda_{image} = Q_G\lambda : \lambda_{image} \in Im(G^T) \}$
Since $Ker(G)$ is orthogonal to $Im(G^T)$, we can multiple $P_G$ in the fisrt equation and multiple by $Q_G$ the second one:

\begin{equation}
P_G(F\lambda + G^{T}\alpha) = P_GF\lambda = P_Gd \\
Q_GG{\lambda} = Q_Ge \\
{\lambda_{image}} = G^T(GG^T)^{-1}e \\
\lambda = \lambda_{kernel} + \lambda_{image}
\end{equation}

Then the equation for $\lambda_{kernel}$ is:

\begin{equation}
P_GF\lambda_{kernel} = P_G(d - F\lambda_{image}) =  P_G\hat{d} \\
\end{equation}

And  $\alpha$ can be solve by:

\begin{equation}
GG^{T}\alpha = G(d - F\lambda) 
\end{equation}


In [ ]:
# solving the project Dual interface problem
I = np.eye(interface_size,interface_size)
GTG = G.T.dot(G)
GGT =  G.dot(G.T)
GTe = G.T.dot(e)
P = I - G.T.dot(np.linalg.inv(GGT)).dot(G)

# Solving lamda im
lambda_im = G.T.dot(np.linalg.solve(GGT,e))

# Solving lamda kernel
PF = P.dot(F)
dhat = d - F.dot(lambda_im)
Pdhat = P.dot(dhat)
lambda_ker = np.linalg.pinv(PF).dot(Pdhat)

dual_int_lambda = lambda_ker + lambda_im


In [ ]:
# eingen spectron of the operators
w, V = np.linalg.eigh(F)

w_id = np.argsort(-w)
w= w[w_id]
V = V[:,w_id]

cond_num_F = w[0]/w[-1]

pw, pV = np.linalg.eigh(PF)
w_id = np.argsort(-pw)
pw= pw[w_id]
pV = pV[:,w_id]

cond_num_PF = pw[0]/pw[-1]

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))
plt.tight_layout(pad=4.0, w_pad=4.5, h_pad=4.0)

ax1.set_ylim([0.0,np.max(w)])
ax1.bar(np.arange(len(w)),w)
ax1.set_title('Eigenvalue of F, $\kappa = $ ' + '{0:.2f}'.format(cond_num_F))


ax2.set_ylim([0.0,np.max(pw)])
ax2.bar(np.arange(len(pw)),pw)
ax2.set_title('Eigenvalue of Project F')
ax2.set_title('Eigenvalue of Project F, $\kappa = $ ' + '{0:.2f}'.format(cond_num_PF))

plt.show()

In [ ]:
global_lambda = np.array(global_lambda).flatten()
residual = abs(global_lambda - dual_int_lambda )

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
plt.tight_layout(pad=4.0, w_pad=4.5, h_pad=5.0)

axs[0].bar(np.arange(len(residual)),residual)
axs[0].set_title('Error per DoFs')

axs[1].scatter(global_lambda, dual_int_lambda )
p , residuals, rank, singular_values, rcond = np.polyfit(global_lambda , dual_int_lambda, 1, full=True)
m = p[0]
b = p[1]
X_plot = np.linspace(axs[1].get_xlim()[0],axs[1].get_xlim()[1],100)
axs[1].plot(X_plot, m*X_plot + b, '-')

correlation = np.corrcoef(global_lambda, dual_int_lambda)[0,1]
axs[1].set_title('Correlation between Primal and Dual Assembly')

axs[1].text(-50.0, 0.0, r"$r^2 = $" + '{0:.2f}'.format(correlation**2) , fontsize=20)

fig.suptitle('Error between Primal and Projected Dual Interface Assembly')
plt.show()

Solving Equilibrium with The Projected Dual Interface with Conjugate Gradient (PCG)
--------------------------------------------------------------------------------------

The Projected Interface equilibrium can be solve iterativaly by a Conjungate Gradient algorithm. The step are the following:

1. Initialize with $r^0 = \hat{d},  \lambda_{ker}^{0} = \tilde{0}, \beta^1 = 0 $

2. Irerate $k = 1,2..,n$ until $ \lVert w^k \rVert > tol $  
    Project residual $w^{k-1} = P_Gr^{k-1}$  
    Precondition $z^{k-1} = \bar{F^{-1}}w^{k-1}$  
    Project  $y^{k-1} = P_Gz^{k-1}$   
    $\beta^{k} = (y^{k-1})^Tw^{k-1}/(y^{k-2})^Tw^{k-2}$  
    $p^{k} = y^{k-1} + \beta^{k}p^{k-1}  $  
    $\alpha^{k} = (y^{k-1})^Tw^{k-1}/(p^{k})^TFp^{k}$  
    $\lambda_{ker}^{k} = \lambda_{ker}^{k-1} + \alpha^{k}p^{k}$  
    $r^{k} = r^{k-1} + \alpha^{k}Fp^{k}$  

3. Set solution $\lambda_{ker} = \lambda_{ker}^{k} $  


In [ ]:
def PCGP(F,dhat,lambda_ker,Precond=None,tol = 1.e-6,max_int = 500):
    # step 1
    beta = 0.0
    rk = dhat
    interface_size = lambda_ker.shape[0]
    lambda_ker = np.zeros(interface_size)
    
    if Precond is None:
        Precond = np.eye(interface_size,interface_size)

    #step 2
    yk2 = np.zeros(interface_size)
    yk1 = np.zeros(interface_size)
    wk1 = np.zeros(interface_size)

    res = []
    for k in range(max_int):
        wk = P.dot(rk)

        norm_wk = np.linalg.norm(wk)
        res.append(norm_wk)
        if norm_wk<tol:
            print('PCG has converged after %i' %(k+1))
            break

        zk = Precond.dot(wk)
        yk = P.dot(zk)

        if k>1:
            #calc beta
            beta = yk.T.dot(wk)/yk1.T.dot(wk1)
        else:
            pk1 = yk

        pk = yk + beta*pk1
        Fpk = F.dot(pk)
        ak = yk.T.dot(wk)/pk.T.dot(Fpk)
        lambda_ker = lambda_ker + ak*pk
        rk = rk - ak*Fpk

        # set n - 1 data
        yk1 = yk
        pk1 = pk
        wk1 = wk


    lambda_pgc = lambda_ker + lambda_im
    return lambda_pgc, res

lambda_pgc, res = PCGP(F,dhat,lambda_ker)

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))    
ax2.scatter(dual_int_lambda,lambda_pgc)    
ax2.set_xlabel('$\lambda$ by Dual Problem')
ax2.set_ylabel('$\lambda$ by PCG')

p , residuals, rank, singular_values, rcond = np.polyfit(dual_int_lambda ,lambda_pgc, 1, full=True)
m = p[0]
b = p[1]
X_plot = np.linspace(ax2.get_xlim()[0],ax2.get_xlim()[1],100)
ax2.plot(X_plot, m*X_plot + b, '-')



ax1.plot(res,'o')    
ax1.set_xlabel('iteration')
ax1.set_ylabel('Projected error $||w^k||$')

fig.suptitle('PCGP without precondicioning.')
plt.show()

In [ ]:
ii_id = sub1.local_interior_dofs_list
bb_id = sub1.local_interface_dofs_list

K1 = sub1.stiffness
Kbb = K[bb_id,:][:,bb_id].todense()
Kii = K[ii_id,:][:,ii_id].todense()
Kbi = K[bb_id,:][:,ii_id].todense()
Kib = Kbi.T
Kii_inv = np.linalg.pinv(Kii)
print(sub1.local_interior_dofs_list)
print(sub1.local_interface_dofs_list)

Sbb = Kbb - Kbi.dot(Kii_inv).dot(Kib)

In [ ]:
ub = [1.0]*len(bb_id)

In [ ]:
ub

In [ ]:
Sbb = Kbb
print(Sbb.shape)
#Kii_inv = np.linalg.inv(Kii)
#Sbb = Kbb - Kbi.dot(Kii_inv).dot(Kib)
#print(Sbb.shape)

In [ ]:
diag_inv = 1.0/Kii.diagonal()
Kii_inv = np.diag(diag_inv .A1)


In [ ]:
S1 = sub1.assemble_primal_schur_complement()
S2 = sub2.assemble_primal_schur_complement()
print(S1.shape)
print(S2.shape)
print(K1.shape)
print(K2.shape)
print(B1.shape)
print(B2.shape)


In [ ]:
from scipy.sparse import bmat
A = np.matrix([[1,2],[2,4]])
B = np.matrix([[3,7],[2,5]])
#A = np.zeros([2,2])
C = bmat([[A, None], [None, B]])

In [ ]:
C.todense()

In [ ]:
Cd = C.diagonal()
print(Cd)
Cd_inv = 1.0/Cd
print(Cd_inv)

Sbb = scipy.sparse.diags(Cd_inv)

In [ ]:
Sbb

In [ ]:
K = sub1.stiffness

In [ ]:
K.shape

In [ ]:
Kii = K[ii_id,:][:,ii_id]
Kbb = K[bb_id,:][:,bb_id]